In [1]:
#dependencies:
import pandas as pd
import time #allows for sleep method. To-Do: Research if there is a method that doesn't require an additional module
from bs4 import BeautifulSoup #html parser
import pymongo #allows direct interaction with mongo db through Python

#required for splinter (To-Do: need to verify splinter dependencies)
from webdriver_manager.chrome import ChromeDriverManager
from splinter import Browser

In [2]:
#create driver
executable_path = {'executable_path':ChromeDriverManager().install()}  #To-do: continue searching for more efficient method
#set up browser
browser = Browser('chrome', **executable_path, headless=False)
#path provided by instruction
news_path = 'https://mars.nasa.gov/news/'
browser.visit(news_path)
time.sleep(3)
html = browser.html
browser.quit()

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389


[WDM] - Driver [C:\Users\Test\.wdm\drivers\chromedriver\win32\89.0.4389.23\chromedriver.exe] found in cache


In [3]:
#use bs4 to parse the title and paragraph
soup=BeautifulSoup(html, 'html.parser') #returns a soup object
news_title = soup.find('h3', class_='').text #works correctly 2021-03-17.
# Observation that most recent news article headline is under the first 
# <h3> tag with no CSS class assigned

In [4]:
#find first slide, which represents most recent news story. Then find
# a div tag with class_ = "article_teaser_body" within that 

news_paragraph =soup.find('li', class_='slide') \
                    .find('div', class_="article_teaser_body").text

In [5]:
print(f'{news_title} \n {news_paragraph}')

NASA's Mars Helicopter to Make First Flight Attempt Sunday 
 The small rotorcraft’s “Wright brothers moment” is two Mars days away.


In [ ]:
#Syntax example from bs4 docs
# for link in soup.find_all('a'):
#     print(link.get('href'))

In [6]:
#set up browser, seems to need to be redone after a browser.quit()
browser = Browser('chrome', **executable_path, headless=False)
gallery_url = 'https://www.jpl.nasa.gov/images?search=&category=Mars'  #pulled from readme.md instructions

browser.visit(gallery_url)
time.sleep(5)
html = browser.html
browser.quit()

In [7]:
soup=BeautifulSoup(html, 'html.parser') #returns a soup object

In [8]:
soup.find('div', class_="SearchResultCard").find('a', class_="group cursor-pointer block").get('href')

'/images/australe-mensa-false-color'

In [9]:
top_photo_url = 'https://jpl.nasa.gov'+soup.find('div', class_="SearchResultCard").find('a', class_="group cursor-pointer block").get('href')

In [10]:
browser = Browser('chrome', **executable_path, headless=False)
browser.visit(top_photo_url)
time.sleep(5)
html = browser.html
browser.quit()

In [11]:
soup=BeautifulSoup(html, 'html.parser')

In [12]:
#find "Download JPG " (trailing space included) label and get preceding href, which should be the full size photo
featured_image_url = soup.find(string='Download JPG ').find_previous('a').get('href')

In [13]:
print(featured_image_url)

https://d2pn8kiwq2w21t.cloudfront.net/original_images/jpegPIA24556.jpg


In [14]:
mars_facts_url = 'https://www.space-facts.com/mars/'

In [15]:
browser = Browser('chrome', **executable_path, headless=False)
browser.visit(mars_facts_url)
# time.sleep(3)
html = browser.html
browser.quit()

In [16]:
soup=BeautifulSoup(html, 'html.parser')

In [17]:
#snippet from: https://stackoverflow.com/questions/23377533/python-beautifulsoup-parsing-table retrieved 2021-03-24

labels = []
points = []

table = soup.find('table', class_='tablepress tablepress-id-p-mars')
table_body = table.find('tbody')
rows = table_body.find_all('tr')
for row in rows:
    cols=row.find_all('td', class_='column-1')
    for col in cols:
        txt = col.get_text()
        labels.append(txt)
    
    cols=row.find_all('td', class_='column-2')
    for col in cols:
        txt = col.get_text()
        points.append(txt)

df = pd.DataFrame({'Mars':points}, index=labels)
html_table = df.to_html()

In [18]:
hemisphere_image_urls = [
    {"title": "Valles Marineris Hemisphere", "img_url": "https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg"},
    {"title": "Cerberus Hemisphere", "img_url": "https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg"},
    {"title": "Schiaparelli Hemisphere", "img_url": "https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg"},
    {"title": "Syrtis Major Hemisphere", "img_url": "https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg"},
    ]

In [35]:
scrape_dict = {}
scrape_dict['news_headline'] = news_title
scrape_dict['news_teaser']   = news_paragraph
scrape_dict['featured_image_url'] = featured_image_url
scrape_dict['html_table'] = html_table
scrape_dict['hemisphere_image_urls'] = hemisphere_image_urls

In [22]:
#conn string
mongo_conn= 'mongodb://localhost:27017'
client = pymongo.MongoClient(mongo_conn)

In [36]:
# Define database and collection
    #deprecated these two lines
    #db = client.mars_db
    #collection = db.items
collection = client.mars_app.mars

In [48]:
collection.drop()

In [45]:
#Dump the dict into mongo as a document
collection.insert_one(scrape_dict)

In [49]:
scrape = client.mars_app.mars.find()

In [56]:
collection.count_documents({}) == 0

True

In [50]:
for record in scrape:
    print(record)